<a href="https://colab.research.google.com/github/QuantAnalyticsTorch/quant_analytics_torch/blob/main/examples/MultivariateWienerPath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Univariate Wiener Path Construction

In [4]:
import time
import math
import numpy as np
import torch

In [5]:
class UnivariateBrownianBridge():
  def __init__(self, number_time_steps):
    self.number_time_steps = number_time_steps
    number = 3

    self.left_index = torch.zeros(number_time_steps, dtype=int)
    self.right_index = torch.zeros(number_time_steps, dtype=int)
    self.bridge_index = torch.zeros(number_time_steps, dtype=int)
    self.left_weight = torch.zeros(number_time_steps)
    self.right_weight = torch.zeros(number_time_steps)
    self.std_dev = torch.zeros(number_time_steps)

    self._map = torch.zeros(number_time_steps, dtype=int)

    self._map[-1] = 1
    self.bridge_index[0] = number_time_steps - 1
    self.std_dev[0] = torch.sqrt(torch.tensor(1.0) * number_time_steps)
    self.left_weight[0] = 0
    self.right_weight[0] = 0

    j=0
    for i in range(1,number_time_steps):
      while self._map[j] == True:
        j = j + 1
      k = j
      while self._map[k] == False:
        k = k + 1
      l = j+((k-1-j)>>1)
      self._map[l]=i
      self.bridge_index[i]=l
      self.left_index[i]=j
      self.right_index[i]=k
      self.left_weight[i]=(k-l)/(k+1-j)
      self.right_weight[i]=(1+l-j)/(k+1-j)
      self.std_dev[i]=np.sqrt(((1+l-j)*(k-l))/(k+1-j))
      j=k+1
      if j>=number_time_steps:
        j=0
        
  @torch.jit.script
  def buildPath(path, z, number: int, number_time_steps: int, left_index, right_index, bridge_index, left_weight, right_weight, std_dev):
    path[-1] = std_dev[0]*z[0]
    j = 0
    k = 0
    l = 0
    i = 0
    for i in range(1,number_time_steps):
      j = left_index[i]
      k = right_index[i]
      l = bridge_index[i]
      lw = left_weight[i]
      rw = right_weight[i]
      sd = std_dev[i]
      if j > 0:
        path[l] = path[j-1] * lw + path[k] * rw + z[i] * sd
      else:
        path[l] = right_weight[i] * path[k] + std_dev[i] * z[i]

In [6]:
brownian = UnivariateBrownianBridge(8)

In [7]:
print(UnivariateBrownianBridge.buildPath.code)

def buildPath(path: Tensor,
    z: Tensor,
    number: int,
    number_time_steps: int,
    left_index: Tensor,
    right_index: Tensor,
    bridge_index: Tensor,
    left_weight: Tensor,
    right_weight: Tensor,
    std_dev: Tensor) -> None:
  _0 = torch.mul(torch.select(std_dev, 0, 0), torch.select(z, 0, 0))
  _1 = torch.copy_(torch.select(path, 0, -1), _0, False)
  _2 = torch.__range_length(1, number_time_steps, 1)
  for _3 in range(_2):
    i = torch.__derive_index(_3, 1, 1)
    j = torch.select(left_index, 0, i)
    _4 = annotate(int, j)
    k = torch.select(right_index, 0, i)
    _5 = annotate(int, k)
    l = torch.select(bridge_index, 0, i)
    _6 = annotate(int, l)
    lw = torch.select(left_weight, 0, i)
    rw = torch.select(right_weight, 0, i)
    sd = torch.select(std_dev, 0, i)
    if torch.gt(_4, 0):
      _7 = torch.select(path, 0, torch.sub(_4, 1))
      _8 = torch.mul(_7, lw)
      _9 = torch.mul(torch.select(path, 0, _5), rw)
      _10 = torch.add(_8, _9, alpha=1)
  

In [8]:
path = torch.zeros(size=(8,2))

x = torch.randn(8,2)

UnivariateBrownianBridge.buildPath(path, x, 2, 8, brownian.left_index, brownian.right_index, brownian.bridge_index, brownian.left_weight, brownian.right_weight, brownian.std_dev)

In [13]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
numberTimeSteps =  128#@param {type:"integer"}
numberSimulation =  200000#@param {type:"integer"}


# First run (includes graph optimization time)
time_start = time.time()
path = torch.zeros(size=(numberTimeSteps,numberSimulation))

brownian = UnivariateBrownianBridge(numberTimeSteps)

sobol_engine =  torch.quasirandom.SobolEngine(numberTimeSteps)

x = sobol_engine.draw(numberSimulation)
y = torch.transpose(torch.erf(x),0,1)
UnivariateBrownianBridge.buildPath(path, y, numberSimulation, numberTimeSteps, brownian.left_index, brownian.right_index, brownian.bridge_index, brownian.left_weight, brownian.right_weight, brownian.std_dev)
time_end = time.time()
time_price_cpu = time_end - time_start
print("First time on a CPU: ", time_price_cpu)

First time on a CPU:  0.6133313179016113


In [ ]:
import torch
import torchvision.models as models
import torch.autograd.profiler as profiler

In [ ]:
numberTimeSteps =  128#@param {type:"integer"}
numberSimulation =  200000#@param {type:"integer"}

path = torch.zeros(size=(numberTimeSteps,numberSimulation), dtype=float)

with profiler.profile(record_shapes=True) as prof:
    with profiler.record_function("univariate_bridge"):
        x = sobol_engine.draw(number)
        y = torch.transpose(torch.erf(x),0,1)
        BrownianBridge.buildPath(path, y, numberSimulation, numberTimeSteps, brownian.left_index, brownian.right_index, brownian.bridge_index, brownian.left_weight, brownian.right_weight, brownian.std_dev)


In [ ]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                         Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
            univariate_bridge         0.18%       1.129ms        99.99%     628.919ms     628.919ms             1  
                    buildPath         7.65%      48.122ms        86.16%     541.890ms     541.890ms             1  
                    aten::mul        55.42%     348.556ms        55.97%     352.013ms     938.701us           375  
                    aten::add        11.89%      74.784ms        16.39%     103.069ms     417.285us           247  
                    aten::erf         5.61%      35.273ms        11.21%      70.532ms      35.266ms             2  
                  aten::copy_         8.91%      56.043ms         8.91% 

# MultivariateBrownian

In [ ]:
numberSimulation = 3
numberTimeSteps = 2
numberStates = 2
dim = numberTimeSteps * numberStates

In [ ]:
sobol_engine =  torch.quasirandom.SobolEngine(dim)
x = sobol_engine.draw(numberSimulation)
x = torch.transpose(x,0,1)

In [ ]:
y = torch.reshape(x,shape=(numberTimeSteps,numberStates,numberSimulation))

In [ ]:
m = torch.zeros(size=(numberStates,numberStates))
m[0,0] = 1
m[1,0] = 1
m[1,1] = 0

In [ ]:
w = torch.matmul(m,y[0,:,:])

In [ ]:
w

tensor([[0.5000, 0.7500, 0.2500],
        [0.5000, 0.7500, 0.2500]])

In [ ]:
class MultivariateBrownianBridge():
  def __init__(self, forward_covariance_matrices):
    self.number_time_steps = len(forward_covariance_matrices)